In [1]:
import numpy as np

p = 71

In [2]:
numbers = [-1, -4, -160, 500]

# List comprehension to compute mod p and print
print([num % p for num in numbers])

[70, 67, 53, 3]


In [3]:
def mod_inverse(a, p):
    return pow(int(a), -1, p)

def mod_fraction(numerator, denominator, p):
    numerator %= p
    denominator %= p
    return (numerator * mod_inverse(denominator, p)) % p

In [4]:
# Define rationals as (numerator, denominator)
def abc_add(a,b,c):
    # Compute elements in finite field
    a_mod = mod_fraction(*a, p)
    b_mod = mod_fraction(*b, p)
    c_mod = mod_fraction(*c, p)

    # Verify a + b = c (in mod field)
    sum_ab_mod = (a_mod + b_mod) % p

    # Print results
    print(f"a (mod {p}): {a_mod}")
    print(f"b (mod {p}): {b_mod}")
    print(f"c (mod {p}): {c_mod}")
    print(f"a + b (mod {p}): {sum_ab_mod}")
    print(f"Verification: {'Pass' if sum_ab_mod == c_mod else 'Fail'}")

In [5]:
a = (5, 6)
b = (11, 12)
c = (21, 12)

abc_add(a,b,c)

a (mod 71): 60
b (mod 71): 66
c (mod 71): 55
a + b (mod 71): 55
Verification: Pass


In [6]:
# Define rationals as (numerator, denominator)
def abc_mul(a,b,c):
    # Compute elements in finite field
    a_mod = mod_fraction(*a, p)
    b_mod = mod_fraction(*b, p)
    c_mod = mod_fraction(*c, p)

    # Verify a * b = c (in mod field)
    sum_ab_mod = (a_mod * b_mod) % p

    # Print results
    print(f"a (mod {p}): {a_mod}")
    print(f"b (mod {p}): {b_mod}")
    print(f"c (mod {p}): {c_mod}")
    print(f"a + b (mod {p}): {sum_ab_mod}")
    print(f"Verification: {'Pass' if sum_ab_mod == c_mod else 'Fail'}")

In [7]:
a = (2, 3)
b = (1, 2)
c = (1, 3)

abc_mul(a,b,c)

a (mod 71): 48
b (mod 71): 36
c (mod 71): 24
a + b (mod 71): 24
Verification: Pass


In [8]:
# Function to compute the determinant of a 2x2 matrix
def determinant(matrix, p):
    a, b, c, d = matrix[0, 0], matrix[0, 1], matrix[1, 0], matrix[1, 1]
    return (a * d - b * c) % p

# Function to compute the modular inverse of a 2x2 matrix
def matrix_inverse_mod(matrix, p):
    a, b, c, d = matrix[0, 0], matrix[0, 1], matrix[1, 0], matrix[1, 1]
    
    # Compute determinant
    det = determinant(matrix, p)
    
    # Find modular inverse of the determinant
    det_inv = mod_inverse(det, p)
    
    # Calculate adjugate matrix (cofactor matrix transposed)
    adjugate = np.array([[d, -b], [-c, a]], dtype=int)
    
    # Apply modular arithmetic to the adjugate
    matrix_inv = (det_inv * adjugate) % p
    
    # Ensure all elements are in [0, p)
    matrix_inv = np.mod(matrix_inv, p)
    
    return matrix_inv

In [9]:
# Example matrix as a NumPy array
matrix = np.array([[1, 1], [1, 4]], dtype=int)

# Compute the inverse
inverse_matrix = matrix_inverse_mod(matrix, p)

# Multiply original matrix and its inverse
result = (np.dot(matrix, inverse_matrix) % p)

# Print results
print("Original matrix:")
print(matrix)
print("\nInverse matrix (mod 71):")
print(inverse_matrix)
print("\nProduct of matrix and its inverse (mod 71):")
print(result)

Original matrix:
[[1 1]
 [1 4]]

Inverse matrix (mod 71):
[[25 47]
 [47 24]]

Product of matrix and its inverse (mod 71):
[[1 0]
 [0 1]]


In [10]:
# Target value
target = 12

# Brute-force approach to find modular square root
solutions = []
for x in range(p):
    if (x * x) % p == target:
        solutions.append(x)

# Print results
print(f"Modular square roots of {target} modulo {p}: {solutions}")

# Verify solutions
for x in solutions:
    print(f"Verification: {x}^2 % {p} = {(x * x) % p}")

Modular square roots of 12 modulo 71: [15, 56]
Verification: 15^2 % 71 = 12
Verification: 56^2 % 71 = 12


In [11]:
def modular_sqrt(a, p):
    """Find a modular square root of a modulo p using Tonelli-Shanks."""
    # Step 1: Check if a is a quadratic residue
    if pow(a, (p - 1) // 2, p) != 1:
        raise ValueError(f"{a} is not a quadratic residue modulo {p}")
    
    # Step 2: Factor p-1 as q * 2^s
    s = 0
    q = p - 1
    while q % 2 == 0:
        q //= 2
        s += 1

    # Step 3: Find a quadratic non-residue z
    for z in range(2, p):
        if pow(z, (p - 1) // 2, p) == p - 1:
            break
    
    # Initialize variables
    m = s
    c = pow(z, q, p)
    t = pow(a, q, p)
    r = pow(a, (q + 1) // 2, p)
    
    # Step 4: Iterate to find the solution
    while t != 1:
        t2 = t
        i = 0
        while t2 != 1:
            t2 = pow(t2, 2, p)
            i += 1
        b = pow(c, 2 ** (m - i - 1), p)
        m = i
        c = pow(b, 2, p)
        t = (t * b * b) % p
        r = (r * b) % p
    
    return r, p - r

# Example usage
p = 71
a = 12
try:
    sqrt1, sqrt2 = modular_sqrt(a, p)
    print(f"Modular square roots of {a} modulo {p}: {sqrt1}, {sqrt2}")
    # Verification
    print(f"Verification: {sqrt1}^2 % {p} = {(sqrt1 * sqrt1) % p}")
    print(f"Verification: {sqrt2}^2 % {p} = {(sqrt2 * sqrt2) % p}")
except ValueError as e:
    print(e)

Modular square roots of 12 modulo 71: 15, 56
Verification: 15^2 % 71 = 12
Verification: 56^2 % 71 = 12


In [12]:
import galois

# Define the Galois field GF(p)
GF = galois.GF(p)

In [13]:
# Define polynomials P(x) and Q(x) over GF(p)
P = galois.Poly([52, 24, 61], field=GF)
Q = galois.Poly([40, 40, 58], field=GF)  # Format Q(x) = 40x^2 + 40x + 58

In [14]:
# Find roots of P(x) and Q(x)
roots_P = P.roots()
roots_Q = Q.roots()

# Compute P(x) + Q(x) and P(x)*Q(x)
P_plus_Q = P + Q
P_times_Q = P * Q

# Find roots of P(x) + Q(x) and P(x)*Q(x)
roots_P_plus_Q = P_plus_Q.roots()
roots_P_times_Q = P_times_Q.roots()

In [15]:
# Print results
print(f"Polynomial P(x): {P}")
print(f"Roots of P(x) (mod {p}): {roots_P}")

print(f"Polynomial Q(x): {Q}")
print(f"Roots of Q(x) (mod {p}): {roots_Q}")

print(f"Polynomial P(x) + Q(x): {P_plus_Q}")
print(f"Roots of P(x) + Q(x) (mod {p}): {roots_P_plus_Q}")

print(f"Polynomial P(x) * Q(x): {P_times_Q}")
print(f"Roots of P(x) * Q(x) (mod {p}): {roots_P_times_Q}")

Polynomial P(x): 52x^2 + 24x + 61
Roots of P(x) (mod 71): [34 42]
Polynomial Q(x): 40x^2 + 40x + 58
Roots of Q(x) (mod 71): []
Polynomial P(x) + Q(x): 21x^2 + 64x + 48
Roots of P(x) + Q(x) (mod 71): [46 49]
Polynomial P(x) * Q(x): 21x^4 + 58x^3 + 26x^2 + 69x + 59
Roots of P(x) * Q(x) (mod 71): [34 42]


In [16]:
def find_line(points, p):
    """Find the equation of a line f(x) = ax + b passing through given points mod p."""
    (x1, y1), (x2, y2) = points
    
    # Calculate slope a = (y2 - y1) / (x2 - x1) mod p
    delta_y = (y2 - y1) % p
    delta_x = (x2 - x1) % p
    a = (delta_y * mod_inverse(delta_x, p)) % p
    
    # Calculate intercept b = y1 - a * x1 mod p
    b = (y1 - a * x1) % p
    
    return a, b

In [17]:
# Define the points
points = [(10, 15), (23, 29)]

# Find the line equation
a, b = find_line(points, p)

# Print the line equation
print(f"The equation of the line is: f(x) = ({a} * x + {b}) mod {p}")

The equation of the line is: f(x) = (12 * x + 37) mod 71


In [18]:
def verify_line(a, b, points, p):
    """Verify that the line f(x) = ax + b passes through the points mod p."""
    for x, y in points:
        assert (a * x + b) % p == y, f"Failed at point ({x}, {y})"
    print("Verification passed for all points!")

In [19]:
# Verify the line
verify_line(a, b, points, p)

Verification passed for all points!


In [20]:
def interpolate_polynomial(p, coordinates, verbosity=False):
    """
    Perform Lagrange interpolation over GF(p) and return the interpolating polynomial.
    
    Parameters:
        p (int): Modulus defining the finite field GF(p).
        coordinates (list of tuple): List of (x, y) points to interpolate.
        verbosity (bool): Whether to print the interpolating polynomial. Default is False.
    
    Returns:
        galois.Poly: The interpolating polynomial over GF(p).
    """
    # Define the finite field
    GF = galois.GF(p)
    
    # Separate x and y coordinates
    x_points, y_points = zip(*coordinates)
    x_points = GF(x_points)
    y_points = GF(y_points)
    
    # Perform Lagrange interpolation
    polynomial = galois.lagrange_poly(x_points, y_points)
    
    # Print polynomial if verbosity is True
    if verbosity:
        print(f"Interpolating polynomial: {polynomial}")
    
    return polynomial

In [21]:
# Define the test function
def test_interpolate_polynomial():
    """
    Test the interpolate_polynomial function.
    """
    p = 71
    coordinates = [(0, 1), (1, 2), (2, 1)]
    expected_degree = len(coordinates) - 1  # Polynomial degree should match
    polynomial = interpolate_polynomial(p, coordinates, verbosity=True)
    
    # Verify degree
    assert polynomial.degree == expected_degree, "Incorrect polynomial degree."
    
    # Verify points
    GF = galois.GF(p)
    for x, y in coordinates:
        assert polynomial(GF(x)) == GF(y), f"f({x}) != {y}"
    print("Test passed.")

In [22]:
test_interpolate_polynomial()

Interpolating polynomial: 70x^2 + 2x + 1
Test passed.
